In [ ]:
#Login to the user's google driver account
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#add the GPG key.
# adding the secret key to the set of trusted keys
!sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys E298A3A825C0D65DFD57CBB651716619E084DAB9
#add repository
# add r-project repository to apt repository
!sudo add-apt-repository 'deb https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/'
# update apt repository
!sudo apt-get update
#install R
!sudo apt install r-base
#install codonw
!sudo apt-get install codonw
#installation of required packages
!echo "install.packages(c('protr','rDNAse','seqinr','matrixTests'))" > installPackages.R
!Rscript installPackages.R
#install biopython
!pip install biopython
!pip install rdkit
!pip install -q condacolab

Executing: /tmp/apt-key-gpghome.0COGMktcfG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys E298A3A825C0D65DFD57CBB651716619E084DAB9
gpg: key 51716619E084DAB9: "Michael Rutter <marutter@gmail.com>" 1 new signature
gpg: Total number processed: 1
gpg:         new signatures: 1
Repository: 'deb https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/'
Description:
Archive for codename: focal-cran40/ components: 
More info: https://cloud.r-project.org/bin/linux/ubuntu
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/archive_uri-https_cloud_r-project_org_bin_linux_ubuntu-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/archive_uri-https_cloud_r-project_org_bin_linux_ubuntu-jammy.list
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cud

In [ ]:
#necessary dependencies
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from os import listdir
import pandas as pd
import numpy as np
import sys
import re
import csv
import condacolab

In [ ]:
condacolab.install()

In [ ]:
#iFeature package
!pip3 install iFeatureOmegaCLI

In [ ]:
#organization code to process here
organism_abbr = ["dde"]

# Section in charge of the recovery of sequences in Kegg

In [ ]:
#cloning of the gitlab repository which contains the script for retrieving the sequences
!git clone https://gitlab.com/fofack/extract_feature_and_sequences.git
#Changing the current directory
%cd extract_feature_and_sequences/
#execution of the sequence extraction script
for i in range (len(organism_abbr)):
  !bash extract_organism_sequence.sh {organism_abbr[i]}
#Changing the current directory
%cd ../..

Cloning into 'extract_feature_and_sequences'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 283 (delta 28), reused 275 (delta 25), pack-reused 0
Receiving objects: 100% (283/283), 18.11 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/extract_feature_and_sequences


NameError: name 'organism_abbr' is not defined

# Section that contains the various functions useful for preprocessing

In [ ]:
#function to retrieve the elements of a directory
def find_dir( path_to_dir):
    dirnames = listdir(path_to_dir)
    return [ dirname for dirname in dirnames if dirname.endswith]

In [ ]:
#function to retrieve fasta files from a directory
def find_fasta_filenames( path_to_dir, suffix=".fasta" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [ ]:
#cleaning of gene sequences to remove sequences <=30 characters and descriptions in front of identifiers
def parsing_sequence_dna(path_input, path_out):
    with open(path_input) as handle:
        with open(path_out, 'a') as clean_file:
            for record in SeqIO.parse(handle, "fasta"):
                if(len(record.seq) > 30):
                    clean_file.write('>'+ "%s" %record.id[4:].upper()+'\n')
                    clean_file.write("%s" %record.seq+'\n')

In [ ]:
#cleanup of protein sequences to remove sequences <=30 characters and descriptions in front of identifiers
def parsing_sequence_protein(path_input, path_out):
    with open(path_input) as handle:
        with open(path_out, 'a') as clean_file:
            for record in SeqIO.parse(handle, "fasta"):
                if(len(record.seq) > 30):
                    clean_file.write('>'+ "%s" %record.id[4:].upper()+'\n')
                    clean_file.write("%s" %record.seq+'\n')

In [ ]:
#cleaning of sequences with certain codons that do not correspond to any amino acid
def delete_sequence_id(path_input, path_out,tab_id):
    with open(path_input) as handle:
        with open(path_out, 'a') as clean_file:
            for record in SeqIO.parse(handle, "fasta"):
                operator = record.id in tab_id
                if(operator == False):
                    clean_file.write('>'+ "%s" %record.id+'\n')
                    clean_file.write("%s" %record.seq+'\n')

In [ ]:
#fusion of the two functions of cleaning gene and protein sequences
def main_parsing_sequences(path_dir_input,path_dir_out):
    path_data = find_fasta_filenames(path_dir_input)
    for i in range(len(path_data)):
        if(path_data[i] == "dna_sequence.fasta"):
            print(path_dir_out+path_data[i])
            parsing_sequence_dna(path_dir_input+path_data[i],path_dir_out+"/"+"dna_sequence.fasta")
        else:
            parsing_sequence_protein(path_dir_input+path_data[i],path_dir_out+"/"+"protein_sequence.fasta")

In [ ]:
# function to retain only sequences that are in both genes and proteins
def clean_sequences(gene_sequence, protein_sequence,path_out):
    #load dataset
    #aa
    fastaSequences = SeqIO.parse(open(protein_sequence), 'fasta')
    seqaaDict_E=dict()
    for fasta in fastaSequences:
        name, sequence = fasta.id, str(fasta.seq)
        seqaaDict_E[name]=sequence
    #print(seqaaDict_E)
    #aa
    fastaSequences = SeqIO.parse(open(gene_sequence), 'fasta')
    seqntDict_E=dict()
    for fasta in fastaSequences:
        name, sequence = fasta.id, str(fasta.seq)
        seqntDict_E[name]=sequence
        #print(seqntDict_E)
    index=0
    fileSeqAA=open(path_out+"EssentialsequenceAA-clean.fasta", 'a')
    fileSeqNT=open( path_out+"EssentialsequenceNT-clean.fasta", 'a')
    for cle in seqntDict_E.keys():
        if cle in seqaaDict_E:
            #print(cle)
            sequenceProt=seqaaDict_E[cle].upper()
            sequenceGene=seqntDict_E[cle].upper()
            gene_Locus=cle
            recordAA=SeqRecord(Seq(sequenceProt), id=str(gene_Locus),description="")
            recordNT=SeqRecord(Seq(sequenceGene), id=str(gene_Locus),description="")
            SeqIO.write(recordAA, fileSeqAA, "fasta")
            SeqIO.write(recordNT, fileSeqNT, "fasta")
            index=index+1


In [ ]:
#function to transform an array of array into a simple array
def transform_in_simple_array(tab):
    probleme_codonw = []
    p = re.compile("[A-Z]+[0-9]*[\_]?[A-Z]*[0-9]+[\_]?[0-9]*[\-]?[A-Z]*[0-9]*")
    for i in range (len(tab)):
        if(len(p.findall(tab[i]))):
            probleme_codonw.append(p.findall(tab[i]))
    id_probleme = np.array(probleme_codonw).flatten()
    return id_probleme

##Feature Methode

In [ ]:
import iFeatureOmegaCLI as iF

In [ ]:
def get_feature_dna(path_input_sequence, path_out):
    dna_descriptors = ["NAC","ANF","ENAC","MMI","LPDF","EIIP","DPCP","TPCP",
                       "Kmer type 1","RCKmer type 1","PseEIIP","PseKNC",
                       "PCPseDNC","PCPseTNC","SCPseDNC","SCPseTNC","PSTNPss",
                       "PSTNPds","CKSNAP type 1", "DAC","DACC", "DCC", "EGAAC",
                       "TAC", "TACC", "TCC",]
    dna = iF.iDNA(path_input_sequence)
    for i in range (len(dna_descriptors)):
        dna.get_descriptor(dna_descriptors[i])
        if(dna_descriptors[i] == "Kmer type 1"):
          protein.to_csv(path_out+"EssentialFeature_collection_"+"Kmer1"+".csv",index=True, header=True)
        elif (dna_descriptors[i] == "RCKmer type 1"):
          protein.to_csv(path_out+"EssentialFeature_collection_"+"RCKmer1"+".csv",index=True, header=True)
        elif (dna_descriptors[i] == "CKSNAP type 1"):
          protein.to_csv(path_out+"EssentialFeature_collection_"+"CKSNAP"+".csv",index=True, header=True)
        else:
          dna.to_csv(path_out+"EssentialFeature_collection_"+dna_descriptors[i]+".csv",index=True, header=True)

In [ ]:
def get_feature_peptide(path_file_sequence, path_out):
    protein_descriptors = ["CTDC","CTDD","CTDT","DDE","EGAAC","GAAC","AC","CC",
                           "ACC","CKSAAP type 1","KSCTriad","DistancePair",
                           "CKSAAGP type 1","GDPC type 1","GTPC type 1","ASDC",
                           "DPC type 1","DPC type 2","ZScale", "AAIndex",
                           "TPC type 1","TPC type 2", "ASDC"]
    protein = iF.iProtein(path_file_sequence)
    for i in range (len(protein_descriptors)):
      protein.get_descriptor(protein_descriptors[i])
      if(protein_descriptors[i] == "CKSAAP type 1"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"CKSAAP"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "DPC type 1"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"DPC1"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "DPC type 2"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"DPC2"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "TPC type 1"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"TPC1"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "TPC type 2"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"TPC2"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "GDPC type 1"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"GDPC1"+".csv",index=True, header=True)
      elif (protein_descriptors[i] == "GTPC type 1"):
        protein.to_csv(path_out+"EssentialFeature_collection_"+"GTPC1"+".csv",index=True, header=True)
      else:
        protein.to_csv(path_out+"EssentialFeature_collection_"+protein_descriptors[i]+".csv",index=True, header=True)

In [ ]:
def main_iFeature(input_dna, out_dna, input_peptide, out_peptide):
    get_feature_dna(input_dna, out_dna)
    get_feature_peptide(input_peptide, out_peptide)

In [ ]:
def split_data(path_dir, path_out):
    file = find_dir(path_dir)
    #file.remove('.ipynb_checkpoints')
    for i in range (len(file)):
        print(file[i])
        with open(path_dir+file[i],"r") as handle:
            data = csv.reader(handle)
            rows = [r for r in data]
            rows[0].pop(0)
            with open(path_out+file[i],"w") as f:
                writer = csv.writer(f)
                for j in range(len(rows)):
                    writer.writerow(rows[j])

# Section responsible for feature extraction using R language packages

In [ ]:
def main(path_input, path_out):
    organism = find_dir(path_input)
    for i in range (len(organism)):
        print("################"+" "+"Feature de l'organisme"+" "+organism[i]+" "+"################")
        os.mkdir(path_out+organism[i])
        os.mkdir("/content/results"+organism[i])
        main_parsing_sequences(path_input+organism[i]+"/", path_out+organism[i]+"/")
        clean_sequences(path_out+organism[i]+"/"+"dna_sequence.fasta", path_out+organism[i]+"/"+"protein_sequence.fasta", path_out+organism[i]+"/")
        chemin = path_out+organism[i]
        pb = !codonw {chemin}/EssentialsequenceNT-clean.fasta results/{organism[i]}/EssentialcodonwFeature.out results/{organism[i]}/EssentialcodonwFeature.blk -nomenu -nowarn -all_indices -human
        id_probleme = transform_in_simple_array(pb)
        print(id_probleme)
        if(len(id_probleme) > 0):
            !mkdir clean_sequence_copy
            !mkdir clean_sequence_copy/{organism[i]}
            !mv /content/sequences_clean/{organism[i]}/dna_sequence.fasta clean_sequence_copy/{organism[i]}/
            !mv /content/sequences_clean/{organism[i]}/protein_sequence.fasta clean_sequence_copy/{organism[i]}/
            !rm /content/sequences_clean/{organism[i]}/EssentialsequenceNT-clean.fasta
            !rm /content/sequences_clean/{organism[i]}/EssentialsequenceAA-clean.fasta
            !rm /content/results/{organism[i]}/EssentialcodonwFeature.out
            !rm /content/results/{organism[i]}/EssentialcodonwFeature.blk
            delete_sequence_id("/content/clean_sequence_copy/"+organism[i]+"/"+"dna_sequence.fasta", path_out+organism[i]+"/"+"dna_sequence.fasta", id_probleme)
            delete_sequence_id("/content/clean_sequence_copy/"+organism[i]+"/"+"protein_sequence.fasta", path_out+organism[i]+"/"+"protein_sequence.fasta", id_probleme)
            clean_sequences(path_out+organism[i]+"/"+"DNA_sequence.fasta", path_out+organism[i]+"/"+"protein_sequence.fasta", path_out+organism[i]+"/")
            chemin = path_out+organism[i]
            pb1 =!codonw {chemin}/EssentialsequenceNT-clean.fasta results/{organism[i]}/EssentialcodonwFeature.out results/{organism[i]}/EssentialcodonwFeature.blk -nomenu -nowarn -all_indices -human
            print(pb1)
            print("************* Debut extraction des features de l'organisme"+" "+organism[i] +"**********")
            #extract sequence feature
            !Rscript /content/extract_feature_and_sequences/gepoFeatureEngineering/scriptFeatureBySeq.R {path_out+organism[i]}/EssentialsequenceNT-clean.fasta {path_out+organism[i]}/EssentialsequenceAA-clean.fasta  /content/results/{organism[i]}/Essential
            !rm /content/results/{organism[i]}/EssentialcodonwFeature.out
            !rm /content/results/{organism[i]}/EssentialcodonwFeature.blk

            #iFeature
            mkdir /content/results_iFeat_tmp
            main_iFeature('/content/sequences_clean/'+organism[i]+'/EssentialsequenceNT-clean.fasta','/content/results_iFeat_tmp/','/content/sequences_clean/'+organism[i]+'/EssentialsequenceAA-clean.fasta','/content/results_iFeat_tmp/')
            split_data('/content/results_iFeat_tmp/', '/content/results/'+organism[i]+'/')
            !sudo rm -r /content/results_iFeat_tmp

            print("************* Fin extraction des features de l'organisme"+" "+organism[i] +"**********")
        else:
            print("************* Debut extraction des features de l'organisme"+" "+organism[i] +"**********")
            #extract sequence feature
            !Rscript /content/extract_feature_and_sequences/gepoFeatureEngineering/scriptFeatureBySeq.R {path_out+organism[i]}/EssentialsequenceNT-clean.fasta {path_out+organism[i]}/EssentialsequenceAA-clean.fasta /content/results/{organism[i]}/Essential

            #iFeature
            mkdir /content/results_iFeat_tmp
            main_iFeature('/content/sequences_clean/'+organism[i]+'/EssentialsequenceNT-clean.fasta','/content/results_iFeat_tmp/','/content/sequences_clean/'+organism[i]+'/EssentialsequenceAA-clean.fasta','/content/results_iFeat_tmp/')
            split_data('/content/results_iFeat_tmp/', '/content/results/'+organism[i]+'/')
            !sudo rm -r /content/results_iFeat_tmp

            !rm /content/results/{organism[i]}/EssentialcodonwFeature.out
            !rm /content/results/{organism[i]}/EssentialcodonwFeature.blk
            print("************* Fin extraction des features de l'organisme"+" "+organism[i] +"**********")


# execution of the main function for the generation of the organism's features



In [ ]:
main("/content/extract_feature_and_sequences/organism/", "/content/sequences_clean/")

# Dataset labeling

In [ ]:
def main_labeling(epath_file, data_csv):
    df = pd.read_csv(data_csv)
    # Labelissage du dataset
    df['essentiality'] = -1
    id_kegg = df.index
    epath = pd.read_excel(epath_file, sheet_name='Sheet1')
    epath['Gene_Locus'] = epath['Gene_Locus'].str.upper()
    epath.set_index("Gene_Locus", inplace=True)
    gene_locus = epath.index
    for i in range (len(id_kegg)):
        if(id_kegg[i] in np.array(gene_locus)):
                df.at[id_kegg[i],'essentiality'] = epath.at[id_kegg[i],'Gene_essentaility']
    df.drop(df[df.essentiality == -1].index, inplace=True)
    mappings = {'NE': 0, 'E': 1}
    classes = df.pop('essentiality')
    essential_labels = classes.map(mappings)
    df['essentiality_status'] = essential_labels
    df = df.reset_index(drop=True)
    features = df.columns[:-1]
    target = ["essentiality_status"]
    df_pr = preprocessing.normalize(df[features])
    df_d = pd.DataFrame(df_pr, columns = features)
    target_f = pd.DataFrame(df["essentiality_status"], columns = target)
    dataset_norm = pd.concat([df_d, target_f], axis=1)
    return dataset_norm

In [ ]:
data_epath = find_dir("/content/extract_feature_and_sequences/Data_epath/")
for i in range (len(organism_abbr)):
  !mkdir /content/{organism_abbr[i]}
  for j in range (len(data_epath)):
    if(organism_abbr[i] == data_epath[j].split("_")[0]):
      organism_data = find_dir("/content/results/"+organism_abbr[i])
      for item in range (len(organism_data)):
        dataset = main_labeling("/content/extract_feature_and_sequences/Data_epath/"+data_epath[j],
                                "/content/results/"+organism_abbr[i]+"/"+organism_data[item])
        subGroupName = organism_data[item].split("_")[2].split(".")[0]
        dataset.to_csv(organism_abbr[i]+"/"+subGroupName+".csv")
        files.download(subGroupName+".csv")




In [7]:
!cd /content
!mkdir dataLabels
# !mkdir
# !cd data
!git clone https://gitlab.com/Berthoazeye/data.git


mkdir: cannot create directory ‘dataLabels’: File exists
fatal: destination path 'data' already exists and is not an empty directory.


In [1]:
import pandas as pd
import os

input_path = 'data/'
output_path = 'dataLabels/'
headers = ['Org','Gene_Locus','KO_Nb', 'Gene_Name', 'Gene_Function', 'E_Score','P_Score','Organism']

len(os.listdir(input_path))

4166

In [2]:
def supprimer_extension(nom_fichier):
    nom_base = os.path.basename(nom_fichier)
    base, _ = os.path.splitext(nom_base)
    return base

In [ ]:
for filename in os.listdir(input_path):
    print("reste---------------------------------------------------------"+str(len(os.listdir(input_path))))
    f = os.path.join(input_path, filename)
    print(f)
    if os.path.isfile(f):
        file_csv = pd.read_csv(f)
        for val in range(0,len(file_csv)):
            # print(file_csv.loc[val,'E_Score'])
            e_score = float(file_csv.loc[val,'E_Score'])
            file_csv.replace(to_replace ="9.16E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="4.28E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="3.47E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="5.20E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="9.33E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="4.31E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="2.64E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="3.23E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="5.82E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="6.11E-",
                 value = "0.000",
                  inplace = True)
            file_csv.replace(to_replace ="9.59E-",
                 value = "0.000",
                  inplace = True)
            # print(file_csv.loc[val,'P_Score'])
            p_score = float(file_csv.loc[val,'P_Score'])
            # print(p_score)
            if (p_score!=0 and e_score!=0):
                if e_score>=0.6:
                    file_csv.loc[val, 'Gene_essentaility'] = 'E'
                else:
                    file_csv.loc[val, 'Gene_essentaility'] = 'NE'
            else:
                if (p_score>=0.03 or e_score>=0.6):
                    file_csv.loc[val, 'Gene_essentaility'] = 'E'
                else:
                    file_csv.loc[val, 'Gene_essentaility'] = 'NE'
    #file_csv.to_csv(f)
    # !rm /content/results/{organism[i]}/EssentialcodonwFeature.out
    !rm {input_path}{filename}
    file_csv.to_excel(output_path+supprimer_extension(filename)+'.xlsx', index= None, header=True)

reste---------------------------------------------------------4166
data/psn_Pseudopedobacter_saltans.csv
reste---------------------------------------------------------4165
data/duc_Dehalococcoides_sp._UCH007.csv
reste---------------------------------------------------------4164
data/mrd_Methylobacterium_radiotolerans.csv
reste---------------------------------------------------------4163
data/msb_Mycobacterium_smegmatis_MC2_155.csv
reste---------------------------------------------------------4162
data/bup_Buchnera_aphidicola_TLW03_Acyrthosiphon_pisum).csv
/usr/bin/sh: 1: Syntax error: ")" unexpected
reste---------------------------------------------------------4162
data/hpx_Helicobacter_pylori_83.csv
reste---------------------------------------------------------4161
data/ahd_Aeromonas_hydrophila_YL17.csv
reste---------------------------------------------------------4160
data/spht_Sphingobium_sp._TKS.csv
reste---------------------------------------------------------4159
data/zmn_Zymomon

reste---------------------------------------------------------4100
data/gba_Gemmatirosa_kalamazoonesis.csv
reste---------------------------------------------------------4099
data/pck_Candidatus_Pantoea_carbekii_US.csv
reste---------------------------------------------------------4098
data/rsc_Ralstonia_solanacearum_CFBP2957.csv
reste---------------------------------------------------------4097
data/lam_Lactobacillus_amylovorus_GRL_1112.csv
reste---------------------------------------------------------4096
data/dmc_Dehalococcoides_mccartyi_BTF08.csv
reste---------------------------------------------------------4095
data/bang_Bifidobacterium_angulatum.csv
reste---------------------------------------------------------4094
data/tgr_Thioalkalivibrio_sulfidiphilus.csv
reste---------------------------------------------------------4093
data/csw__Chlamydia_trachomatis_Sweden2.csv
reste---------------------------------------------------------4092
data/tcl_Thauera_chlorobenzoica.csv
reste--------

reste---------------------------------------------------------4031
data/tnu_Thermococcus_nautili.csv
reste---------------------------------------------------------4030
data/bud_Burkholderia_sp_Bp5365.csv
reste---------------------------------------------------------4029
data/tao_Thioalkalimicrobium_aerophilum.csv
reste---------------------------------------------------------4028
data/dsh_Dinoroseobacter_shibae.csv
reste---------------------------------------------------------4027
data/thf_Thermofilum_uzonense.csv
reste---------------------------------------------------------4026
data/hpyj_Helicobacter_pylori_oki898.csv
reste---------------------------------------------------------4025
data/nfa_Nocardia_farcinica_IFM_10152.csv
reste---------------------------------------------------------4024
data/edh_Escherichia_coli_DH1.csv
reste---------------------------------------------------------4023
data/ftr_Francisella_tularensis_subsp_tularensis_NE061598.csv
reste-----------------------------

reste---------------------------------------------------------3966
data/llt_Lactococcus_lactis_subsp._lactis_CV56.csv
reste---------------------------------------------------------3965
data/tti_Thioalkalivibrio_thiocyanoxidans.csv
reste---------------------------------------------------------3964
data/mek_Methanosarcina_sp._Kolksee.csv
reste---------------------------------------------------------3963
data/xpe_Xanthomonas_perforans.csv
reste---------------------------------------------------------3962
data/mgh_Mycoplasma_gallisepticum_R(high).csv
/usr/bin/sh: 1: Syntax error: "(" unexpected
reste---------------------------------------------------------3962
data/ptt_Photorhabdus_temperata.csv
reste---------------------------------------------------------3961
data/slr_Salinispira_pacifica.csv
reste---------------------------------------------------------3960
data/smi_Sinorhizobium_meliloti_Rm41.csv
reste---------------------------------------------------------3959
data/mcac_Mycoplasma_ca

reste---------------------------------------------------------3900
data/slc_Streptomyces_lydicus.csv
reste---------------------------------------------------------3899
data/acx_Azotobacter_chroococcum.csv
reste---------------------------------------------------------3898
data/tna_Thermotoga_neapolitana.csv
reste---------------------------------------------------------3897
data/sphj_Sphingomonas_sp._JJ-A5.csv
reste---------------------------------------------------------3896
data/vpa_Vibrio_parahaemolyticus_RIMD_2210633.csv
reste---------------------------------------------------------3895
data/pri_Paenibacillus_riograndensis.csv
reste---------------------------------------------------------3894
data/meso_Mesorhizobium_sp._B7.csv
reste---------------------------------------------------------3893
data/cac_Clostridium_acetobutylicum_ATCC_824.csv
reste---------------------------------------------------------3892
data/mhx_Mannheimia_haemolytica_M42548.csv
reste------------------------------

In [ ]:
!git clone https://gitlab.com/Berthoazeye/data.git 

Cloning into 'data'...
remote: Enumerating objects: 4647, done.
remote: Counting objects: 100% (4647/4647), done.
remote: Compressing objects: 100% (1486/1486), done.
remote: Total 4647 (delta 3436), reused 4367 (delta 3160), pack-reused 0
Receiving objects: 100% (4647/4647), 65.83 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (3436/3436), done.
Updating files: 100% (4627/4627), done.


In [29]:
!cd /content/dataLabels

# !git config --global user.email "romeoberthoazeye@gmail.com"
# !git config --global user.name "berthoazeyeh"

# !git init
# !git add .
# !git commit -m "data"
# !git branch -M main
!git remote add origin1 git@github.com:berthoazeyeh/EPATHDATAS.git
!git push -u origin1 main

error: remote origin1 already exists.
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [37]:
# !ssh-keygen
# !git push --set-upstream origin main

# !ls /content/.ssh/id_rsa

!git credential-cache exit
!git credential-cache store


!git branch -M master
!git push -u origin master

^C
fatal: could not read Username for 'https://github.com': No such device or address


In [39]:
import zipfile
import os

def compresser_dossier(source_dir, output_zip):
    """
    Compresser les données d'un dossier en un fichier ZIP.

    Args:
        source_dir (str): Chemin du dossier source à compresser.
        output_zip (str): Chemin du fichier ZIP de sortie.

    Returns:
        None
    """
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, source_dir))

# Exemple d'utilisation :
dossier_source = '/content/data'
fichier_zip_sortie = '/content/fichier33.zip'

compresser_dossier(dossier_source, fichier_zip_sortie)
